<a href="https://colab.research.google.com/github/kalkande/recsys-python/blob/main/04_recsys.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [585]:
#第4章 k近傍法
#準備
import pprint
import numpy as np
np.set_printoptions(precision=3)

# 上位K件
TOP_K = 3
# 近傍アイテム数
K_ITEMS = 3
# しきい値
THETA = 0

Du = np.array([
               [5, 3, +1],
               [6, 2, +1],
               [4, 1, +1],
               [8, 5, -1],
               [2, 4, -1],
               [3, 6, -1],
               [7, 6, -1],
               [4, 2, np.nan],
               [5, 1, np.nan],
               [8, 6, np.nan],
               [3, 4, np.nan],
               [4, 7, np.nan],
               [4, 4, np.nan],
])
I = np.arange(Du.shape[0])
x = Du[:,:-1]
ru = Du[:,-1]

Iu = I[~np.isnan(ru)]
Iup = I[ru==+1]
Iun = I[ru==-1]
Iu_not = np.setdiff1d(I, Iu)

In [586]:
# 関数
def dist(xi, xj):
    """
    距離関数：アイテムiの特徴ベクトルxiとアイテムjの特徴ベクトルxjのユークリッド距離を返す。

    Parameters
    ----------
    xi : ndarray
        アイテムiの特徴ベクトル
    xj : ndarray
        アイテムjの特徴ベクトル

    Returns
    -------
    float
        ユークリッド距離
    """
    #【    問01    】
    # ０１ ユークリッド距離
    distance = np.sqrt( np.sum( [(xi[k] - xj[k])**2 for k in range(0,np.size(xi),1) ] ) )
    distance = np.sqrt( np.sum( (xi - xj)**2) )
    

    return distance

In [587]:
i = 7
j = 2
print('dist(x{}, x{}) = {:.3f}'.format(i, j, dist(x[i], x[j])))
i = 7
j = 3
print('dist(x{}, x{}) = {:.3f}'.format(i, j, dist(x[i], x[j])))

dist(x7, x2) = 1.000
dist(x7, x3) = 5.000


In [588]:
#02 アイテム-アイテム距離行列
#【    問02    】
D = np.zeros( (np.size(Iu_not),np.size(Iu)) )


for i in range(0,np.size(Iu_not),1):
  for j in range(0,np.size(Iu),1):
    D[i,j] = dist(x[Iu_not[i]],x[Iu[j]])

D = np.array([[dist(x[Iu_not[i]],x[Iu[j]])
          for i in range(0,np.size(Iu_not),1)]  
            for j in range(0,np.size(Iu),1)])

print('D = \n{}'.format(D))

D = 
[[1.414 2.    4.243 2.236 4.123 1.414]
 [2.    1.414 4.472 3.606 5.385 2.828]
 [1.    1.    6.403 3.162 6.    3.   ]
 [5.    5.    1.    5.099 4.472 4.123]
 [2.828 4.243 6.325 1.    3.606 2.   ]
 [4.123 5.385 5.    2.    1.414 2.236]
 [5.    5.385 1.    4.472 3.162 3.606]]


In [589]:
#03 距離の昇順に並べ替えたインデックスの配列
#【    問03    】
D = np.zeros( (np.size(I),np.size(Iu)) )

D = np.array([[dist(x[I[i]],x[Iu[j]])
          for i in range(0,np.size(I),1)]  
            for j in range(0,np.size(Iu),1)])

Ii = np.argsort(D.T,axis=1)
print('Ii = \n{}'.format(Ii))

Ii = 
[[0 1 2 4 3 5 6]
 [1 0 2 3 6 4 5]
 [2 0 1 4 5 3 6]
 [3 6 0 1 5 2 4]
 [4 5 0 2 1 6 3]
 [5 4 0 6 1 2 3]
 [6 3 0 5 1 4 2]
 [2 0 1 4 5 3 6]
 [2 1 0 4 3 5 6]
 [3 6 0 1 5 4 2]
 [4 5 0 2 1 6 3]
 [5 6 4 0 3 1 2]
 [0 4 5 1 2 6 3]]


In [590]:
#04 近傍k件のアイテムのインデックス配列
#【    問04    】
Ii = Ii[:,:3]
print('Ii = \n{}'.format(Ii))

Ii = 
[[0 1 2]
 [1 0 2]
 [2 0 1]
 [3 6 0]
 [4 5 0]
 [5 4 0]
 [6 3 0]
 [2 0 1]
 [2 1 0]
 [3 6 0]
 [4 5 0]
 [5 6 4]
 [0 4 5]]


In [591]:
#05 各対象アイテムの近傍アイテム集合
#【    問05    】
Ii_dic =  { i : Ii[i] for i in Iu_not }
print('Ii = ')
pprint.pprint(Ii_dic)

Ii = 
{7: array([2, 0, 1]),
 8: array([2, 1, 0]),
 9: array([3, 6, 0]),
 10: array([4, 5, 0]),
 11: array([5, 6, 4]),
 12: array([0, 4, 5])}


In [592]:
# 嗜好予測（多数決方式）
# 関数
def predict1(u, i):
    """
    予測関数（多数決方式）：多数決方式によりユーザuのアイテムiに対する予測評価値を返す。

    Parameters
    ----------
    u : int
        ユーザuのID（ダミー）
    i : int
        アイテムiのID

    Returns
    -------
    float
        予測評価値
    """
    #【    問06    】
    Iip = Iup[np.isin(Iup,Ii[i])]
    print('I{}+ = {}'.format(i, Iip))
    #【    問07    】
    Iin = Iun[np.isin(Iun,Ii[i])]
    print('I{}- = {}'.format(i, Iin))
    #【    問08    】
    rui = 0
    if np.size(Iip) > np.size(Iin):
      rui = +1
    elif np.size(Iip) < np.size(Iin):
      rui = -1
    else:
      0
    return rui

In [593]:
u = 0
i = 7
print('predict1({}, {}) = {:.3f}'.format(u, i, predict1(u, i)))
u = 0
i = 9
print('predict1({}, {}) = {:.3f}'.format(u, i, predict1(u, i)))

I7+ = [0 1 2]
I7- = []
predict1(0, 7) = 1.000
I9+ = [0]
I9- = [3 6]
predict1(0, 9) = -1.000


In [600]:
# 嗜好予測（平均方式）
# 関数
def predict2(u, i):
    """
    予測関数（平均方式）：平均方式によりユーザuのアイテムiに対する評価値を予測する。

    Parameters
    ----------
    u : int
        ユーザuのID（ダミー）
    i : int
        アイテムiのID

    Returns
    -------
    float
        予測評価値
    """
#    【    問09    】
    rui = np.sum( [ ru[k] for k in Ii[i] ] ) / np.size(Ii[i])
    return rui

In [601]:
u = 0
i = 7
print('predict2({}, {}) = {:.3f}'.format(u, i, predict2(u, i)))
u = 0
i = 9
print('predict2({}, {}) = {:.3f}'.format(u, i, predict2(u, i)))

predict2(0, 7) = 1.000
predict2(0, 9) = -0.333


In [596]:
# 09 平均方式による予測評価値
def score(u, i):
    """
    スコア関数：ユーザuのアイテムiに対するスコアを返す。

    Parameters
    ----------
    u : int
        ユーザuのID
    i : int
        アイテムiのID

    Returns
    -------
    float
        スコア
    """
    return predict2(u, i)

In [627]:
def order(u, I):
    """
    順序付け関数：アイテム集合Iにおいて、ユーザu向けの推薦リストを返す。

    Parameters
    ----------
    u : int
        ユーザuのID
    I : ndarray
        アイテム集合

    Returns
    -------
    list
        タプル(アイテムID: スコア)を要素にした推薦リスト
    """
    scores = {i: score(u, i) for i in I}
    #【    問10    】
    {scores.pop(k) for k, v in list(scores.items()) if v < THETA}
    rec_list = sorted(scores.items(), key=lambda x:x[1], reverse=True)[:TOP_K]
    return rec_list

In [628]:
# コード
u = 0
rec_list = order(u, Iu_not)
print('rec_list = ')
for i, scr in rec_list:
    print('{}: {:.3f}'.format(i, scr))

rec_list = 
7: 1.000
8: 1.000
